<div style="border-radius:10px; border:#D0C2F0 solid; padding: 15px; background-color: #F8F1E8; font-size:150%; text-align:left">

<h3 align="left"><font color='#5E5273'>📄 Veri Setinin Hikayesi </font></h3>
    <center><img src="https://i.imgur.com/6QiRhzo.png"> </center>
    
*   Veri Amazonun bir e-kitap servisi olan, Kindle Store'un kullanıcılar tarafından kitaplara yapılan yorumlarından oluşmaktadır.
*   Site adresi : <a href="https://www.amazon.com/Kindle-Store/b?ie=UTF8&node=133140011"> Kindle Store </a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/kaggle/input/kindle-reviews/kindle_reviews.csv")
df.head(5)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>👀 Features: </font></h3>
    

1. **Unnamed: 0:** ikinci bir index stunu, veride fazlalık, çıkarılabilir.

2. **asin:** Bu sütun, ASIN (Amazon Standard Identification Number), Amazon'un ürünleri benzersiz bir şekilde tanımlamak için kullandığı bir sistemdir, kısacası ürün ID si diyebiliriz.

3. **helpful:** Bu sütun, bir incelemenin diğer kullanıcılar tarafından yararlı bulunup bulunmadığını gösterir.[Evet,Toplam]

4. **overall:** Bu sütun, incelemeyi yapan kişinin verdiği puanı ifade eder.

5. **reviewText:** Bu sütun, kullanıcının yazdığı incelemeyi içerir. 

6. **reviewTime:** Bu sütun, incelemenin yazıldığı tarihi içerir.

7. **reviewerID:** Bu sütun, incelemeyi yazan kullanıcının ID sini içerir.

8. **reviewerName:** Bu sütun, incelemeyi yazan kullanıcının adını içerir.

9. **summary:** Bu sütun, incelemenin özetini içerir.

10. **unixReviewTime:** Bu sütun, incelemenin yazıldığı tarihin Unix zaman damgasını içerir.


In [ ]:
df = df[["asin","helpful","overall","reviewText","unixReviewTime","reviewerID","reviewerName"]]

In [ ]:
df.head()

In [ ]:
df["Date"] = pd.to_datetime(df["unixReviewTime"], unit="s")
df.head()

In [ ]:
df.shape

# <p style="background-color:#F8F1E8; font-family:newtimeroman;color:#602F44; font-size:150%; text-align:center; border-radius: 15px 50px;"> 🙄 Rating Products ⭐</p>

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Ürün Puanlarının Ortalaması</p>

In [ ]:
df[df["asin"] == "B000F83SZQ"]

In [ ]:
df.groupby("asin").agg(mean_rating=("overall", "mean")).reset_index()

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Ürünlere yapılan yorumların puanlarının ortalamasını alan basit bir sıralama yönteminin bazı sakıncaları bulunmaktadır. Bu yöntem, detaylı bir analiz yapma yeteneğine sahip olmadığı için belirli trendleri kaçırabilir. 

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Puan Zamanlarına Göre Ağırlıklı Ortalama</p>

In [ ]:
a = df[df["asin"] == "B000FA64PK"]
a

In [ ]:
weighted_rating = a['overall'][(a["day_diff"]<=30)].mean() * 28/100 + \
a['overall'][(a["day_diff"]>30) & (a["day_diff"]<=90)].mean() * 26/100 + \
a['overall'][(a["day_diff"]>90) & (a["day_diff"]<=180)].mean() * 24/100 + \
a['overall'][(a["day_diff"]>180)].mean() * 22/100
weighted_rating

In [ ]:
max_date = df["Date"].max()
max_date

In [ ]:
df["day_diff"] = (max_date - df["Date"]).dt.days
df.head()

In [ ]:
def apply_penalty(on:pd.Series, dependency:pd.Series, bins:list, weights:list) -> pd.Series:
    """
    This function applies a penalty on ratings based on a measurement range.

    Parameters:
    - on: pandas Series, representing the ratings.
    - dependency: pandas Series, representing the measurements.
    - bins: list defining dependency ranges, for example: [0, 30, 90, 180]
    - weights: list defining penalties, for example: [100, 98, 95, 90]

    Returns:
    A pandas Series with applied penalties on ratings.
    """
    
    if (len(weights) - len(bins)) != 0: raise Exception("The weight_bins must be same range as day_bins")
    
    bins.append(np.inf)
    bins[0] = -np.inf
    
    penalties = pd.cut(dependency, bins=bins, labels=np.array(weights)/100).astype(float)
    point = on * penalties
    
    return point

In [ ]:
df["overall_weighted_by_day"] = apply_penalty(on = df["overall"],
                                              dependency = df["day_diff"],
                                              bins = df["day_diff"].quantile([0, 0.25, 0.50, 0.75]).to_list(),
                                              weights = [100, 90, 85, 80])
df.head(5)

In [ ]:
df.groupby("asin").agg(mean_rating=("overall", "mean"),
                        weighted_mean_rating=("overall_weighted_by_day", "mean"))

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Ürüne verilen puanlardan, yakın dönemde verilenleri hiç veya az cezalandırırken, eski dönemlerde verilen puanları daha yüksek bir şekilde cezalandırma uygularsak, mevcut trendleri yakalayabiliriz.

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Kullanıcı Temelli Ağırlıklı Ortalama</p>

In [ ]:
(df["reviewerID"].value_counts() > 500).sum()

In [ ]:
(df["reviewerID"] == "A1F6404F1VG29J").sum()

In [ ]:
df["reviewer_count_by_user"] = df.groupby("reviewerID")["reviewerID"].transform("count")
df.head()

In [ ]:
df["reviewer_count_by_user"].quantile([0, 0.30, 0.60, 0.90]).to_list()

In [ ]:
df["user_based_weighted_rate"] = apply_penalty(on = df["overall"],
                                              dependency = df["reviewer_count_by_user"],
                                              bins = df["reviewer_count_by_user"].quantile([0, 0.30, 0.60, 0.90]).to_list(),
                                              weights = [80, 90, 95, 100])
df.head(5)

In [ ]:
df.groupby("asin").agg(mean_rating=("overall", "mean"),
                        weighted_mean_rating=("overall_weighted_by_day", "mean"),
                      user_weighted_mean_rating=("user_based_weighted_rate", "mean"))

# helpful

In [ ]:
votes = df["helpful"].str.replace("[^\d,]","",regex=True).str.split(',', expand=True).rename(columns={0:"helpful_yes",1:"vote_Total"}).astype(int)
votes["helpful_no"] = votes["vote_Total"] - votes["helpful_yes"]
votes

In [ ]:
df = pd.concat([df,votes],axis=1)
df

In [ ]:
df["helpful_sum_by_user"] = df.groupby("reviewerID")["helpful_yes"].transform("sum")
df.head()

In [ ]:
df["helpful_sum_by_user"].quantile([0, 0.25, 0.50, 0.75]).to_list()

In [ ]:
df["helpful_based_weighted_rate"] = apply_penalty(on = df["overall"],
                                              dependency = df["helpful_sum_by_user"],
                                              bins = df["helpful_sum_by_user"].quantile([0, 0.25, 0.50, 0.75]).to_list(),
                                              weights = [80, 90, 95, 100])
df.head(5)

In [ ]:
df.groupby("asin").agg(mean_rating=("overall", "mean"),
                        weighted_mean_rating=("overall_weighted_by_day", "mean"),
                      user_weighted_mean_rating=("user_based_weighted_rate", "mean"),
                      helpful_based_weighted_rate=("helpful_based_weighted_rate", "mean"))

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

* Bir kullanıcının kitaplara gösterdiği ilgiyi dikkate alırsak ve ürün puanlamasında onun yorumlarına daha fazla ağırlık verirsek, bu sayede tecrübeli kullanıcıların görüşleri diğerlerine göre daha önemli hale getirilebilir.

# <p style="background-color:#F8F1E8; font-family:newtimeroman;color:#602F44; font-size:150%; text-align:center; border-radius: 15px 50px;"> ⬆ Sorting Reviews ⬇ </p>

In [ ]:
df.head(5)

In [ ]:
df["asin"].value_counts()

In [ ]:
reviews_df = df[df["asin"] == "B00BTIDW4S"][["helpful","overall","reviewText","reviewerID"]]
reviews_df.head()

In [ ]:
reviews_df

In [ ]:
votes = reviews_df["helpful"].str.replace("[^\d,]","",regex=True).str.split(',', expand=True).rename(columns={0:"helpful_yes",1:"vote_Total"}).astype(int)
votes["helpful_no"] = votes["vote_Total"] - votes["helpful_yes"]
votes

In [ ]:
reviews_df = pd.concat([reviews_df,votes],axis=1)
reviews_df

In [ ]:
def score_pos_neg_diff(col1,col2):
    return col1 - col2

def score_average_rating(col1, col2):
    if col1 + col2 == 0:
        return 0
    return col1 / (col1 + col2)

def wilson_lower_bound(up, down, confidence=0.95):
    import scipy.stats as st
    import math
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [ ]:
reviews_df["score_pos_neg_diff"] = reviews_df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"],
                                                                                 x["helpful_no"]), axis=1)
reviews_df.sort_values(by="score_pos_neg_diff", ascending=False).head(10)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>
    
 **score_pos_neg_diff (Pozitif ve Negatif Puan Farkı)**
   - *Avantajlar:*
     - Basit ve anlaşılır bir metrik.
     - Sadece pozitif ve negatif puanların farkını dikkate alır, bu da belirgin bir **duygu farkı**nı yakalamasına yardımcı olabilir.

   - *Dezavantajlar:*
     - Ortalama değerlendirmeleri dikkate almaz, bu da tüm incelemelerin aynı ağırlığa sahip olduğu anlamına gelir.
     - Puanlar geniş bir aralığa sahipse, bu metrik yetersiz kalabilir.

In [ ]:
reviews_df["score_average_rating"] = reviews_df.apply(lambda x: score_average_rating(x["helpful_yes"],
                                                                                    x["helpful_no"]), axis=1)
reviews_df.sort_values(by="score_average_rating", ascending=False).head(10)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

**score_average_rating (Ortalama Puanlama)**
   - *Avantaj:*
     - Her yoruma eşit derecede yaklaşır.

   - *Dezavantaj:*
     - Düşük etkileşime sahip yorumlar, yüksek etkileşime sahip yorumların önüne geçebilir.


In [ ]:
reviews_df["wilson_lower_bound"] = reviews_df.apply(lambda x: wilson_lower_bound(x["helpful_yes"],
                                                                        x["helpful_no"]), axis=1)
reviews_df.sort_values(by="wilson_lower_bound", ascending=False).head(10)

<div style="border-radius:10px; border:#6B8BA0 solid; padding: 15px; background-color: #F2EADF; font-size:150%; text-align:left">

<h3 align="left"><font color='#6B8BA0'>💭 Comment: </font></h3>

**wilson_lower_bound**
   - *Avantajlar:*
     - İstatistiksel güven aralığına dayandığından, yanlılığı ortadan kaldırır.

   - *Dezavantajlar:*
     - Büyük verilerde hesaplaması diğerlerine göre daha uzun sürebilir.
    
<center> <img src="https://i.imgur.com/1sBrz6R.png" > </center>